In [1]:
!python --version
# Basic packages
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from typing import List, Dict
# Helper modules
from numpy import load
import calendar
import joblib
from sklearn.preprocessing import StandardScaler
plt.rcdefaults()
print(matplotlib.rcParams['axes.prop_cycle'])

Python 3.12.7
cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])


In [2]:
train_path = "CRSP_ConsumerDiscretionary_2000_2015.csv"
test_path = "CRSP_ConsumerDiscretionary_2016_2024.csv"

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

def inspect_dataset(df, name):
    print(f"\n{name} dataset shape: {df.shape}")

    print("\nData types:")
    print(df.dtypes)

    print("\nMissing value ratio:")
    print(df.isnull().mean().sort_values(ascending=False))

    print("\nUnique value count per column:")
    print(df.nunique().sort_values())

    print("\nSample unique values for object columns:")
    for col in df.select_dtypes(include='object').columns:
        print(f"{col}: {df[col].dropna().unique()[:5]}")

    print("\nSummary statistics for numeric columns:")
    print(df.describe(include=[float, int]))

inspect_dataset(df_train, "Training")
inspect_dataset(df_test, "Test")


/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_15133/3882517259.py:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(train_path)
/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_15133/3882517259.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(test_path)



Training dataset shape: (4880065, 11)

Data types:
PERMNO      int64
date       object
EXCHCD      int64
SICCD       int64
TICKER     object
COMNAM     object
PERMCO      int64
PRC       float64
VOL       float64
RET        object
SHROUT    float64
dtype: object

Missing value ratio:
PRC       0.025926
VOL       0.025921
TICKER    0.025124
RET       0.000804
SHROUT    0.000409
PERMNO    0.000000
date      0.000000
EXCHCD    0.000000
SICCD     0.000000
COMNAM    0.000000
PERMCO    0.000000
dtype: float64

Unique value count per column:
EXCHCD          9
SICCD         285
PERMCO       3208
PERMNO       3265
TICKER       3620
COMNAM       3913
date         4029
SHROUT      63473
PRC         97215
RET        315833
VOL       1096425
dtype: int64

Sample unique values for object columns:
date: ['2000-01-03' '2000-01-04' '2000-01-05' '2000-01-06' '2000-01-07']
TICKER: ['DGSE' 'DGC' 'GGUY' 'XIOX' 'ATCM']
COMNAM: ['DALLAS GOLD & SILVER EXCHANGE IN' 'D G S E COMPANIES INC'
 'GOOD GUYS INC' 'XI

# Trading Day Coverage Analysis

In [3]:
def summarize_top_mktcap_quality(path, date_start, date_end, top_n=50):
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df[(df['date'] >= date_start) & (df['date'] <= date_end)]
    
    df = df[~df['RET'].isin(['C', 'B'])]
    df['RET'] = df['RET'].astype(float)
    df['PRC'] = df['PRC'].astype(float).abs()
    df['mkt_cap'] = df['PRC'] * df['SHROUT']

    top_permnos = df.groupby('PERMNO')['mkt_cap'].mean().sort_values(ascending=False).head(top_n).index
    top_df = df[df['PERMNO'].isin(top_permnos)].copy()
    total_days = df['date'].nunique()

    summary = top_df.groupby('PERMNO').apply(lambda x: pd.Series({
        'avg_mkt_cap': x['mkt_cap'].mean(),
        'trading_days': x['date'].nunique(),
        'trading_coverage': x['date'].nunique() / total_days,
        'PRC_missing_ratio': x['PRC'].isnull().mean(),
        'VOL_missing_ratio': x['VOL'].isnull().mean(),
        'SHROUT_missing_ratio': x['SHROUT'].isnull().mean(),
        'RET_missing_ratio': x['RET'].isnull().mean()
    }))

    return summary.sort_values(by='avg_mkt_cap', ascending=False)

summary_df = summarize_top_mktcap_quality(
    path="CRSP_ConsumerDiscretionary_2000_2015.csv",
    date_start="2000-01-01",
    date_end="2015-12-31",
    top_n=50
)

print(summary_df.head(10))


/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_15133/3474543099.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


         avg_mkt_cap  trading_days  trading_coverage  PRC_missing_ratio  \
PERMNO                                                                    
10107   2.867913e+08        4025.0          0.999007           0.000000   
55976   2.215097e+08        4025.0          0.999007           0.000000   
14542   1.984949e+08         440.0          0.109208           0.000000   
22111   1.785950e+08          80.0          0.019856           0.000000   
12490   1.685508e+08         491.0          0.121866           0.000000   
90319   1.358228e+08        2862.0          0.710350           0.000000   
10104   1.175756e+08        4025.0          0.999007           0.000000   
13407   1.155092e+08         910.0          0.225862           0.000000   
40483   9.506039e+07         261.0          0.064780           0.003831   
92611   9.400759e+07        1172.0          0.290891           0.000000   

        VOL_missing_ratio  SHROUT_missing_ratio  RET_missing_ratio  
PERMNO                        

/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_15133/3474543099.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = top_df.groupby('PERMNO').apply(lambda x: pd.Series({


In [4]:
def clean_crsp_dataset(path, date_start, date_end, trading_coverage_threshold=0.99):
    df = pd.read_csv(path)

    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    num_raw_stocks = df['PERMNO'].nunique()
    print("Number of raw stocks before cleaning:", num_raw_stocks)

    df = df[(df['date'] >= date_start) & (df['date'] <= date_end)]

    df = df[~df['RET'].isin(['C', 'B'])]
    df['RET'] = df['RET'].astype(float)

    df['PRC'] = df['PRC'].astype(float).abs()

    total_days = df['date'].nunique()

    stock_days = df.groupby('PERMNO')['date'].nunique()

    valid_permnos = stock_days[stock_days >= total_days * trading_coverage_threshold].index
    df = df[df['PERMNO'].isin(valid_permnos)]
    
    missing_summary = df[['PRC', 'VOL', 'SHROUT']].isnull().sum()
    print("\nMissing values before dropping:")
    print(missing_summary)
    
    df = df.dropna(subset=['PRC', 'VOL', 'SHROUT'])

    df = df[df['EXCHCD'].isin([1, 2, 3])]
    
    return df

train_clean = clean_crsp_dataset(
    path="CRSP_ConsumerDiscretionary_2000_2015.csv",
    date_start="2000-01-01",
    date_end="2015-12-31"
)

test_clean = clean_crsp_dataset(
    path="CRSP_ConsumerDiscretionary_2016_2024.csv",
    date_start="2016-01-01",
    date_end="2024-12-31"
)

train_clean.to_csv("CRSP_2000_2015_cleaned.csv", index=False)
test_clean.to_csv("CRSP_2016_2024_cleaned.csv", index=False)
print("Train stocks:", train_clean['PERMNO'].nunique(), "Test stocks:", test_clean['PERMNO'].nunique())

train_clean = pd.read_csv("CRSP_2000_2015_cleaned.csv", parse_dates=["date"])
test_clean = pd.read_csv("CRSP_2016_2024_cleaned.csv", parse_dates=["date"])

def final_check(df, name):
    print(f"\n{name} dataset check")

    print("\nMissing values:")
    print(df.isnull().sum())

    print("\nData types:")
    print(df.dtypes)

    print("\nDuplicated rows:", df.duplicated().sum())

    duplicates = df.duplicated(subset=['PERMNO', 'date']).sum()
    print("Duplicated [PERMNO, date] pairs:", duplicates)

    ret_zero_ratio = df.groupby("PERMNO")["RET"].apply(lambda x: (x == 0).mean())

    threshold = 0.3
    flagged_stocks = ret_zero_ratio[ret_zero_ratio > threshold]
    return flagged_stocks

train_flagged = final_check(train_clean, "Train")
test_flagged = final_check(test_clean, "Test")

print(f"\nTotal flagged stocks (Train): {len(train_flagged)}")
print(train_flagged.sort_values(ascending=False).head(10))

print(f"\nTotal flagged stocks (Test): {len(test_flagged)}")
print(test_flagged.sort_values(ascending=False).head(10))


/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_15133/3064706608.py:14: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Number of raw stocks before cleaning: 3265

Missing values before dropping:
PRC       253
VOL       256
SHROUT      6
dtype: int64


/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_15133/3064706608.py:14: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Number of raw stocks before cleaning: 1346

Missing values before dropping:
PRC       1550
VOL       1550
SHROUT       0
dtype: int64
Train stocks: 338 Test stocks: 395

Train dataset check

Missing values:
PERMNO    0
date      0
EXCHCD    0
SICCD     0
TICKER    0
COMNAM    0
PERMCO    0
PRC       0
VOL       0
RET       0
SHROUT    0
dtype: int64

Data types:
PERMNO             int64
date      datetime64[ns]
EXCHCD             int64
SICCD              int64
TICKER            object
COMNAM            object
PERMCO             int64
PRC              float64
VOL              float64
RET              float64
SHROUT           float64
dtype: object

Duplicated rows: 0
Duplicated [PERMNO, date] pairs: 0

Test dataset check

Missing values:
PERMNO    0
date      0
EXCHCD    0
SICCD     0
TICKER    0
COMNAM    0
PERMCO    0
PRC       0
VOL       0
RET       0
SHROUT    0
dtype: int64

Data types:
PERMNO             int64
date      datetime64[ns]
EXCHCD             int64
SICCD              in

# Stock Selection Strategy

In [5]:
def select_top50_from_train_and_match_test(
    train_df, test_df, cutoff_date="2015-12-31", top_n=50, coverage_threshold=0.99
):
    """
    Steps:
    1. Select the top_n stocks by market cap on the last day of the training set.
    2. Check if these stocks meet the coverage threshold in the test set.
    3. If not enough stocks meet the threshold, continue to add more until top_n is reached.

    Returns:
    - train_top: Training set data (only top_n stocks)
    - test_top: Test set data (only top_n stocks)
    - final_permnos: List of selected PERMNOs (length = top_n)
    """

    train_df['date'] = pd.to_datetime(train_df['date'])
    test_df['date'] = pd.to_datetime(test_df['date'])

    last_day = pd.Timestamp(cutoff_date)
    cutoff_df = train_df[train_df['date'] == last_day].copy()
    cutoff_df['mkt_cap'] = cutoff_df['PRC'].abs() * cutoff_df['SHROUT']
    ranked_permnos = (
        cutoff_df.groupby('PERMNO')['mkt_cap']
        .mean()
        .sort_values(ascending=False)
    )

    print(f"[Info] Number of stocks ranked by market cap: {len(ranked_permnos)}")

    test_total_days = test_df['date'].nunique()
    test_days_by_permno = test_df.groupby("PERMNO")['date'].nunique()

    final_permnos = []
    for permno in ranked_permnos.index:
        coverage = test_days_by_permno.get(permno, 0) / test_total_days
        if coverage >= coverage_threshold:
            final_permnos.append(permno)
        if len(final_permnos) >= top_n:
            break

    print(f"[Result] Number of successfully matched stocks: {len(final_permnos)}")
    
    train_top = train_df[train_df["PERMNO"].isin(final_permnos)].copy()
    test_top = test_df[test_df["PERMNO"].isin(final_permnos)].copy()

    return train_top, test_top, final_permnos

train_clean = pd.read_csv("CRSP_2000_2015_cleaned.csv", parse_dates=["date"])
test_clean = pd.read_csv("CRSP_2016_2024_cleaned.csv", parse_dates=["date"])

train_top50, test_top50, top_permnos = select_top50_from_train_and_match_test(
    train_clean, test_clean, cutoff_date="2015-12-31", top_n=50
)

train_top50.to_csv("CRSP_2000_2015_top50_cleaned.csv", index=False)
test_top50.to_csv("CRSP_2016_2024_top50_cleaned.csv", index=False)

print(f"Successfully matched {len(top_permnos)} consistent stocks across train & test (>=99% coverage).")


[Info] 市值排名样本数: 336
[Result] 最终成功匹配的股票数量：50
 Successfully matched 50 consistent stocks across train & test (≥99% coverage).


# Verify stock matching results

In [ ]:
def print_final_top50_by_train_mktcap(train_df, test_df, top_permnos):
    """
    Print the final Top 50 stock information (sorted by train set market cap).
    Output columns:
    PERMNO, Train Ticker, Test Ticker, Train MktCap, Found in Test?, Rank
    """

    # Calculate market cap
    train_df['mkt_cap'] = train_df['PRC'] * train_df['SHROUT']
    test_df['mkt_cap'] = test_df['PRC'] * test_df['SHROUT']

    # Average market cap and TICKER
    train_mktcap = train_df.groupby("PERMNO")['mkt_cap'].mean()
    train_ticker = train_df.groupby("PERMNO")['TICKER'].first()
    test_ticker = test_df.groupby("PERMNO")['TICKER'].first()

    # Merge tables (only use train's market cap for sorting)
    df = pd.DataFrame({
        "Train Ticker": train_ticker,
        "Test Ticker": test_ticker,
        "Train MktCap": train_mktcap
    }).loc[top_permnos]

    # Whether successfully matched in test set
    df["Found in Test?"] = df["Test Ticker"].notnull().replace({True: "Yes", False: "No"})

    # Sort by train market cap and assign rank
    df = df.sort_values("Train MktCap", ascending=False)
    df["Rank"] = range(1, len(df) + 1)

    # Output
    print("\n Final Top 50 Stock Alignment Summary (Sorted by Train MktCap):\n")
    print(df[["Train Ticker", "Test Ticker", "Train MktCap", "Found in Test?", "Rank"]]
          .to_string(formatters={"Train MktCap": "{:,.2f}".format}))

print_final_top50_by_train_mktcap(train_top50, test_top50, top_permnos)


 Final Top 50 Stock Alignment Summary (Sorted by Train MktCap):

       Train Ticker Test Ticker   Train MktCap Found in Test?  Rank
PERMNO                                                             
10107          MSFT        MSFT 286,791,277.76            Yes     1
55976           WMT         WMT 221,509,682.96            Yes     2
10104          ORCL        ORCL 117,575,646.85            Yes     3
66181            HD          HD  84,937,210.31            Yes     4
43449           MCD         MCD  62,168,463.49            Yes     5
84788          AMZN        AMZN  59,388,985.09            Yes     6
17005           CVS         CVS  43,649,507.02            Yes     7
19502           WAG         WBA  42,213,913.37            Yes     8
86356          EBAY        EBAY  40,972,449.07            Yes     9
49154            DH         TGT  39,019,445.95            Yes    10
61399           LOW         LOW  38,923,986.97            Yes    11
87055          COST        COST  29,844,583.58    

# Final Dataset Validation

In [7]:
train_top50 = pd.read_csv("CRSP_2000_2015_top50_cleaned.csv", parse_dates=["date"])
test_top50 = pd.read_csv("CRSP_2016_2024_top50_cleaned.csv", parse_dates=["date"])

def check_dataset(df, name):
    print(f"\n Dataset Check: {name}")
    print(f"- Shape: {df.shape}")
    print(f"- Date range: {df['date'].min().date()} to {df['date'].max().date()}")
    print("- Missing values:")
    print(df.isnull().sum())
    print("- Total rows with any NaN:", df.isnull().any(axis=1).sum())
    print("- Duplicate rows:", df.duplicated().sum())
    print("- PERMNO + date duplicates:", df.duplicated(subset=["PERMNO", "date"]).sum())
    print("- Unique PERMNOs:", df['PERMNO'].nunique())
    print("- Unique TICKERs:", df['TICKER'].nunique())

check_dataset(train_top50, "Train Top 50")
check_dataset(test_top50, "Test Top 50")



 Dataset Check: Train Top 50
- Shape: (201218, 11)
- Date range: 2000-01-03 to 2015-12-31
- Missing values:
PERMNO    0
date      0
EXCHCD    0
SICCD     0
TICKER    0
COMNAM    0
PERMCO    0
PRC       0
VOL       0
RET       0
SHROUT    0
dtype: int64
- Total rows with any NaN: 0
- Duplicate rows: 0
- PERMNO + date duplicates: 0
- Unique PERMNOs: 50
- Unique TICKERs: 57

 Dataset Check: Test Top 50
- Shape: (113200, 11)
- Date range: 2016-01-04 to 2024-12-31
- Missing values:
PERMNO    0
date      0
EXCHCD    0
SICCD     0
TICKER    0
COMNAM    0
PERMCO    0
PRC       0
VOL       0
RET       0
SHROUT    0
dtype: int64
- Total rows with any NaN: 0
- Duplicate rows: 0
- PERMNO + date duplicates: 0
- Unique PERMNOs: 50
- Unique TICKERs: 55


# Factor Dataset Analysis

In [9]:
def analyze_factors(file_path: str) -> pd.DataFrame:
    factors = pd.read_csv(file_path, parse_dates=["date"])

    print("Factor dataset check:")
    print("- Shape:", factors.shape)
    print("- Columns:", list(factors.columns))
    print("- Date range:", f"{factors['date'].min().date()} to {factors['date'].max().date()}")

    print("\nMissing values per column:")
    print(factors.isnull().sum())

    dup_dates = factors.duplicated(subset="date").sum()
    print("\nDuplicated dates:", dup_dates)

    print("\nPreview of the first 5 rows:")
    print(factors.head())

    for test_date in ["2000-01-01", "2000-01-02"]:
        day = calendar.day_name[pd.to_datetime(test_date).weekday()]
        print(f"{test_date} is: {day}")

    return factors

df = analyze_factors("5_Factors_Plus_Momentum.csv")

Factor dataset check:
- Shape: (6289, 8)
- Columns: ['date', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd']
- Date range: 2000-01-03 to 2024-12-31

Missing values per column:
date     0
mktrf    0
smb      0
hml      0
rmw      0
cma      0
rf       0
umd      0
dtype: int64

Duplicated dates: 0

Preview of the first 5 rows:
        date   mktrf     smb     hml     rmw     cma       rf     umd
0 2000-01-03 -0.0071 -0.0006 -0.0141 -0.0150 -0.0064  0.00021 -0.0008
1 2000-01-04 -0.0406  0.0033  0.0206  0.0047  0.0145  0.00021 -0.0191
2 2000-01-05 -0.0009  0.0033  0.0016  0.0041  0.0111  0.00021 -0.0049
3 2000-01-06 -0.0073 -0.0004  0.0126  0.0065  0.0121  0.00021 -0.0149
4 2000-01-07  0.0321 -0.0094 -0.0142 -0.0088 -0.0096  0.00021  0.0056
2000-01-01 is: Saturday
2000-01-02 is: Sunday


# Risk-Free Rate Data Alignment Check

In [10]:
test_df = pd.read_csv("CRSP_2016_2024_top50_cleaned.csv", parse_dates=["date"])
factors = pd.read_csv("5_Factors_Plus_Momentum.csv", parse_dates=["date"])

test_df["date"] = pd.to_datetime(test_df["date"]).dt.normalize()
factors["date"] = pd.to_datetime(factors["date"]).dt.normalize()

missing_dates = test_df[~test_df["date"].isin(factors["date"])]["date"].unique()

print("Missing rf trading days (first 10):")
for dt in missing_dates[:10]:
    print(f"{dt.date()} (weekday={dt.weekday()})")

print(f"Total missing dates: {len(missing_dates)}")

缺失 rf 的交易日（前 10 个）:
总共缺失 0 个日期


# Final Training and Testing Datasets

In [12]:
def prepare_excess_return_data(
    train_file: str,
    test_file: str,
    factor_file: str,
    output_train_file: str,
    output_test_file: str,
    output_train_cleaned_file: str,
    output_test_cleaned_file: str
) -> None:

    train_df = pd.read_csv(train_file, parse_dates=["date"])
    test_df = pd.read_csv(test_file, parse_dates=["date"])
    factors = pd.read_csv(factor_file, parse_dates=["date"])

    train_df["date"] = pd.to_datetime(train_df["date"]).dt.normalize()
    test_df["date"] = pd.to_datetime(test_df["date"]).dt.normalize()
    factors["date"] = pd.to_datetime(factors["date"]).dt.normalize()

    factors = factors[["date", "rf"]]

    train_merged = pd.merge(train_df, factors, on="date", how="left")
    test_merged = pd.merge(test_df, factors, on="date", how="left")

    missing_train = train_merged["rf"].isna().sum()
    missing_test = test_merged["rf"].isna().sum()
    print(f"Missing rf in train: {missing_train}, in test: {missing_test}")

    train_merged = train_merged.dropna(subset=["rf"])
    test_merged = test_merged.dropna(subset=["rf"])

    train_merged["EXRET"] = train_merged["RET"] - train_merged["rf"]
    test_merged["EXRET"] = test_merged["RET"] - test_merged["rf"]

    train_merged = train_merged.dropna(subset=["EXRET"])
    test_merged = test_merged.dropna(subset=["EXRET"])
    
    train_merged = train_merged.sort_values(by=["PERMNO", "date"]).reset_index(drop=True)
    test_merged = test_merged.sort_values(by=["PERMNO", "date"]).reset_index(drop=True)

    train_merged.to_csv(output_train_file, index=False)
    test_merged.to_csv(output_test_file, index=False)

    def remove_outliers(group):
        lower = group["EXRET"].quantile(0.01)
        upper = group["EXRET"].quantile(0.99)
        return group[(group["EXRET"] >= lower) & (group["EXRET"] <= upper)]

    train_cleaned = train_merged.groupby("PERMNO").apply(remove_outliers).reset_index(drop=True)
    test_cleaned = test_merged.groupby("PERMNO").apply(remove_outliers).reset_index(drop=True)

    train_cleaned.to_csv(output_train_cleaned_file, index=False)
    test_cleaned.to_csv(output_test_cleaned_file, index=False)

    print("Datasets merged and excess returns calculated successfully.")
    print(f"Original train size: {len(train_merged)}, cleaned train size: {len(train_cleaned)}")
    print(f"Original test size: {len(test_merged)}, cleaned test size: {len(test_cleaned)}")


prepare_excess_return_data(
    "CRSP_2000_2015_top50_cleaned.csv",
    "CRSP_2016_2024_top50_cleaned.csv",
    "5_Factors_Plus_Momentum.csv",
    "CRSP_2000_2015_top50_with_exret.csv",
    "CRSP_2016_2024_top50_with_exret.csv",
    "CRSP_2000_2015_top50_with_exret_cleaned.csv",
    "CRSP_2016_2024_top50_with_exret_cleaned.csv"
)


Missing rf in train: 0, in test: 0


/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_15133/4272901896.py:61: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_cleaned = train_merged.groupby("PERMNO").apply(remove_outliers).reset_index(drop=True)
/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_15133/4272901896.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_cleaned = test_merged.groupby("PERMNO").apply(remove_o

Datasets merged and excess returns calculated successfully.
Original train size: 201218, cleaned train size: 197120
Original test size: 113200, cleaned test size: 110900


In [13]:
df = pd.read_csv("CRSP_2016_2024_top50_with_exret_cleaned.csv", parse_dates=["date"])
df["weekday"] = df["date"].dt.dayofweek
print(df["weekday"].value_counts())

weekday
1    22904
2    22578
4    22378
3    22346
0    20694
Name: count, dtype: int64


# Sliding Window Dataset Construction

# X Standardization with Original Y Scale

In [14]:

def generate_datasets_for_multiple_windows(
    df: pd.DataFrame,
    window_sizes: List[int] = [5, 21, 252, 512],
    prefix: str = "train",
):
    """Generate rolling window datasets for backtesting (past *win* days → predict *t+1* day).

    *Only* the definition of the target y is modified:
        X_t   = [EXRET_{t-win}, …, EXRET_{t-1}]
        y_t+1 =  EXRET_{t+1}

  
    """

    results = {}

    df = df.copy()
    df["PRC"] = df["PRC"].abs()

    df_by_date = {d: g for d, g in df.groupby("date")}

    for win in window_sizes:
        X_list, y_list, meta_list = [], [], []

        for permno, group in df.groupby("PERMNO"):
            group = group.sort_values("date")
            exret = group["EXRET"].values
            dates = group["date"].values
            shroud = group["SHROUT"].values
            prc = group["PRC"].values
            mktcap = prc * shroud

            for i in range(win-1, len(group) - 1):
                
                signal_date = pd.Timestamp(dates[i])
                ret_date    = pd.Timestamp(dates[i + 1])
                current_date  = signal_date
                current_mktcap = mktcap[i]

                peers = df_by_date[current_date]
                all_mktcap = peers["PRC"].abs().values * peers["SHROUT"].values
                rank = (all_mktcap <= current_mktcap).sum()
                mktcap_percentile = rank / len(all_mktcap)

                X_list.append(exret[i - win + 1 : i + 1])
                y_list.append(exret[i + 1])

                meta_list.append(
                    {
                        "PERMNO": permno,
                        "date": signal_date,
                        "ret_date"   : ret_date,
                        "MKTCAP": current_mktcap,
                        "MKTCAP_PERCENTILE": mktcap_percentile,
                    }
                )

        X_arr = np.array(X_list)
        y_arr = np.array(y_list)
        meta_df = pd.DataFrame(meta_list)
        meta_df["signal_month"] = meta_df["date"].dt.to_period("M")
        meta_df["ret_month"]    = meta_df["ret_date"].dt.to_period("M")
        order = meta_df["ret_date"].argsort().values
        X_arr, y_arr = X_arr[order], y_arr[order]
        meta_df = meta_df.iloc[order].reset_index(drop=True)

        results[f"X_{prefix}_{win}"] = X_arr
        results[f"y_{prefix}_{win}"] = y_arr
        results[f"meta_{prefix}_{win}"] = meta_df

        print(
            f" {prefix.upper()} | Window = {win} | X shape = {X_arr.shape} | y shape = {y_arr.shape}"
        )

    return results




def unpack_window_datasets(
    train_datasets: Dict[str, object],
    test_datasets: Dict[str, object],
    window_sizes: List[int] = [5, 21, 252, 512],
):
    for win in window_sizes:
        globals()[f"X_train_{win}"] = train_datasets[f"X_train_{win}"]
        globals()[f"y_train_{win}"] = train_datasets[f"y_train_{win}"]
        globals()[f"meta_train_{win}"] = train_datasets.get(f"meta_train_{win}", None)

        globals()[f"X_test_{win}"] = test_datasets[f"X_test_{win}"]
        globals()[f"y_test_{win}"] = test_datasets[f"y_test_{win}"]
        globals()[f"meta_test_{win}"] = test_datasets.get(f"meta_test_{win}", None)

        print(
            f" Unpacked → X_train_{win}, y_train_{win}, X_test_{win}, y_test_{win}"
        )


def prepare_and_save_window_data(
    train_csv_path: str,
    test_csv_path: str,
    window_sizes: List[int] = [5, 21, 252, 512],
    output_npz_path: str = "all_window_datasets.npz",
    split_date: str = "2016-01-01",
):
    train_df = pd.read_csv(train_csv_path, parse_dates=["date"])
    test_df  = pd.read_csv(test_csv_path , parse_dates=["date"])
    full_df  = pd.concat([train_df, test_df], ignore_index=True).sort_values(["PERMNO","date"])

    full_sets = generate_datasets_for_multiple_windows(full_df, window_sizes, "full")

    save_dict = {}

    for win in window_sizes:
        X_full   = full_sets[f"X_full_{win}"]
        y_full   = full_sets[f"y_full_{win}"]
        meta_full = full_sets[f"meta_full_{win}"]

        mask_train = meta_full["ret_date"] < split_date

        for lab, m in [("train", mask_train), ("test", ~mask_train)]:
            X_part, y_part, meta_part = X_full[m], y_full[m], meta_full[m].reset_index(drop=True)
            save_dict[f"X_{lab}_{win}"]   = X_part
            save_dict[f"y_{lab}_{win}"]   = y_part
            save_dict[f"meta_{lab}_{win}"] = meta_part.to_dict()
            save_dict[f"market_caps_{lab}_{win}"] = meta_part["MKTCAP"].values

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(save_dict[f"X_train_{win}"])
        X_test_scaled  = scaler.transform(save_dict[f"X_test_{win}"])

        save_dict[f"X_train_{win}"] = X_train_scaled
        save_dict[f"X_test_{win}"]  = X_test_scaled
        joblib.dump(scaler, f"scaler_window_{win}.pkl")

        print(f"FINISH  Window={win}  Train={X_train_scaled.shape}  Test={X_test_scaled.shape}")

    np.savez(output_npz_path, **save_dict)
    print(f"\nAll datasets saved safely to: {output_npz_path}")

if __name__ == "__main__":
    prepare_and_save_window_data(
        train_csv_path="CRSP_2000_2015_top50_with_exret_cleaned.csv",
        test_csv_path="CRSP_2016_2024_top50_with_exret_cleaned.csv",
    )


 FULL | Window = 5 | X shape = (307770, 5) | y shape = (307770,)
 FULL | Window = 21 | X shape = (306970, 21) | y shape = (306970,)
 FULL | Window = 252 | X shape = (295420, 252) | y shape = (295420,)
 FULL | Window = 512 | X shape = (282420, 512) | y shape = (282420,)
FINISH  Window=5  Train=(196920, 5)  Test=(110850, 5)
FINISH  Window=21  Train=(196120, 21)  Test=(110850, 21)
FINISH  Window=252  Train=(184570, 252)  Test=(110850, 252)
FINISH  Window=512  Train=(171570, 512)  Test=(110850, 512)

All datasets saved safely to: all_window_datasets.npz


In [16]:
data = np.load("all_window_datasets.npz", allow_pickle=True)

X_train_5 = data["X_train_5"]
y_train_5 = data["y_train_5"]
meta_train_5 = pd.DataFrame(data["meta_train_5"].item())

X_test_5 = data["X_test_5"]
y_test_5 = data["y_test_5"]
meta_test_5 = pd.DataFrame(data["meta_test_5"].item())

print("=== Train Window 5 ===")
print("X_train_5 shape:", X_train_5.shape)
print("y_train_5 shape:", y_train_5.shape)
print("meta_train_5 head:")
print(meta_train_5.head())
print("-" * 50)

print("=== Test Window 5 ===")
print("X_test_5 shape:", X_test_5.shape)
print("y_test_5 shape:", y_test_5.shape)
print("meta_test_5 head:")
print(meta_test_5.head())
print("-" * 50)
print(meta_test_5['date'][:5])
print(y_test_5[:5])

=== Train Window 5 ===
X_train_5 shape: (196920, 5)
y_train_5 shape: (196920,)
meta_train_5 head:
   PERMNO       date   ret_date        MKTCAP  MKTCAP_PERCENTILE signal_month  \
0   44644 2000-01-07 2000-01-10  3.336514e+07           0.883721      2000-01   
1   15456 2000-01-07 2000-01-10  9.109508e+05           0.116279      2000-01   
2   81481 2000-01-07 2000-01-10  3.198164e+06           0.255814      2000-01   
3   77702 2000-01-07 2000-01-10  4.554635e+06           0.348837      2000-01   
4   16678 2000-01-07 2000-01-10  1.575560e+07           0.674419      2000-01   

  ret_month  
0   2000-01  
1   2000-01  
2   2000-01  
3   2000-01  
4   2000-01  
--------------------------------------------------
=== Test Window 5 ===
X_test_5 shape: (110850, 5)
y_test_5 shape: (110850,)
meta_test_5 head:
   PERMNO       date   ret_date       MKTCAP  MKTCAP_PERCENTILE signal_month  \
0   75510 2016-01-04 2016-01-05  45783493.73           0.775510      2016-01   
1   81655 2016-01-04 2016-

# Unscaled Original Data

In [5]:

def generate_datasets_for_multiple_windows(
    df: pd.DataFrame,
    window_sizes: List[int] = [5, 21, 252, 512],
    prefix: str = "train",
):
    """
    Generate rolling window datasets for backtesting (past *win* days → predict *t+1* day).

    Only the definition of target y is changed:
        X_t   = [EXRET_{t-win}, ..., EXRET_{t-1}]
        y_t+1 =  EXRET_{t+1}

    Other fields and output structure remain the same as previous version.
    """

    results = {}

    df = df.copy()
    df["PRC"] = df["PRC"].abs()  # Take absolute value of price to avoid negative sign

    # Cache the table for each date in advance for market cap percentile calculation
    df_by_date = {d: g for d, g in df.groupby("date")}

    for win in window_sizes:
        X_list, y_list, meta_list = [], [], []

        for permno, group in df.groupby("PERMNO"):
            group = group.sort_values("date")
            exret = group["EXRET"].values
            dates = group["date"].values
            shroud = group["SHROUT"].values
            prc = group["PRC"].values
            mktcap = prc * shroud

            for i in range(win-1, len(group) - 1):
                signal_date = pd.Timestamp(dates[i])
                ret_date    = pd.Timestamp(dates[i + 1])
                current_date  = signal_date
                current_mktcap = mktcap[i]

                peers = df_by_date[current_date]
                all_mktcap = peers["PRC"].abs().values * peers["SHROUT"].values
                rank = (all_mktcap <= current_mktcap).sum()
                mktcap_percentile = rank / len(all_mktcap)

                X_list.append(exret[i - win + 1 : i + 1])
                y_list.append(exret[i + 1])

                meta_list.append(
                    {
                        "PERMNO": permno,
                        "date": signal_date,
                        "ret_date": ret_date,
                        "MKTCAP": current_mktcap,
                        "MKTCAP_PERCENTILE": mktcap_percentile,
                    }
                )

        X_arr = np.array(X_list)
        y_arr = np.array(y_list)
        meta_df = pd.DataFrame(meta_list)
        meta_df["signal_month"] = meta_df["date"].dt.to_period("M")
        meta_df["ret_month"]    = meta_df["ret_date"].dt.to_period("M")
        order = meta_df["ret_date"].argsort().values
        X_arr, y_arr = X_arr[order], y_arr[order]
        meta_df = meta_df.iloc[order].reset_index(drop=True)

        results[f"X_{prefix}_{win}"] = X_arr
        results[f"y_{prefix}_{win}"] = y_arr
        results[f"meta_{prefix}_{win}"] = meta_df

        print(
            f"{prefix.upper()} | Window = {win} | X shape = {X_arr.shape} | y shape = {y_arr.shape}"
        )

    return results


def unpack_window_datasets(
    train_datasets: Dict[str, object],
    test_datasets: Dict[str, object],
    window_sizes: List[int] = [5, 21, 252, 512],
):
    for win in window_sizes:
        globals()[f"X_train_{win}"] = train_datasets[f"X_train_{win}"]
        globals()[f"y_train_{win}"] = train_datasets[f"y_train_{win}"]
        globals()[f"meta_train_{win}"] = train_datasets.get(f"meta_train_{win}", None)

        globals()[f"X_test_{win}"] = test_datasets[f"X_test_{win}"]
        globals()[f"y_test_{win}"] = test_datasets[f"y_test_{win}"]
        globals()[f"meta_test_{win}"] = test_datasets.get(f"meta_test_{win}", None)

        print(
            f"Unpacked: X_train_{win}, y_train_{win}, X_test_{win}, y_test_{win}"
        )


def prepare_and_save_window_data(
    train_csv_path: str,
    test_csv_path: str,
    window_sizes: List[int] = [5, 21, 252, 512],
    output_npz_path: str = "all_window_datasets_unscaled.npz",
    split_date: str = "2016-01-01",
):
    train_df = pd.read_csv(train_csv_path, parse_dates=["date"])
    test_df = pd.read_csv(test_csv_path, parse_dates=["date"])
    full_df = pd.concat([train_df, test_df], ignore_index=True).sort_values(["PERMNO", "date"])

    full_sets = generate_datasets_for_multiple_windows(full_df, window_sizes, "full")
    save_dict = {}

    for win in window_sizes:
        X_full = full_sets[f"X_full_{win}"]
        y_full = full_sets[f"y_full_{win}"]
        meta_full = full_sets[f"meta_full_{win}"]

        mask_train = meta_full["ret_date"] < split_date

        for lab, m in [("train", mask_train), ("test", ~mask_train)]:
            X_part = X_full[m]
            y_part = y_full[m]
            meta_part = meta_full[m].reset_index(drop=True)

            save_dict[f"X_{lab}_{win}"] = X_part
            save_dict[f"y_{lab}_{win}"] = y_part
            save_dict[f"meta_{lab}_{win}"] = meta_part.to_dict()
            save_dict[f"market_caps_{lab}_{win}"] = meta_part["MKTCAP"].values

        print(f"FINISH Window={win} Train={X_full[mask_train].shape} Test={X_full[~mask_train].shape}")

    np.savez(output_npz_path, **save_dict)
    print(f"\nAll datasets saved safely to: {output_npz_path}")

if __name__ == "__main__":
    prepare_and_save_window_data(
        train_csv_path="CRSP_2000_2015_top50_with_exret_cleaned.csv",
        test_csv_path="CRSP_2016_2024_top50_with_exret_cleaned.csv",
    )


 FULL | Window = 5 | X shape = (307770, 5) | y shape = (307770,)
 FULL | Window = 21 | X shape = (306970, 21) | y shape = (306970,)
 FULL | Window = 252 | X shape = (295420, 252) | y shape = (295420,)
 FULL | Window = 512 | X shape = (282420, 512) | y shape = (282420,)
FINISH  Window=5  Train=(196920, 5)  Test=(110850, 5)
FINISH  Window=21  Train=(196120, 21)  Test=(110850, 21)
FINISH  Window=252  Train=(184570, 252)  Test=(110850, 252)
FINISH  Window=512  Train=(171570, 512)  Test=(110850, 512)

All datasets saved safely to: all_window_datasets_unscaled.npz


# Scaled Data for N-Beats and Autoformer

In [6]:

def generate_datasets_for_multiple_windows(
    df: pd.DataFrame,
    window_sizes: List[int] = [5, 21, 252, 512],
    prefix: str = "train",
):
    

    results = {}

    df = df.copy()
    df["PRC"] = df["PRC"].abs()

    df_by_date = {d: g for d, g in df.groupby("date")}

    for win in window_sizes:
        X_list, y_list, meta_list = [], [], []

        for permno, group in df.groupby("PERMNO"):
            group = group.sort_values("date")
            exret = group["EXRET"].values
            dates = group["date"].values
            shroud = group["SHROUT"].values
            prc = group["PRC"].values
            mktcap = prc * shroud

            for i in range(win-1, len(group) - 1):
                
                signal_date = pd.Timestamp(dates[i])      
                ret_date    = pd.Timestamp(dates[i + 1])
                current_date  = signal_date
                current_mktcap = mktcap[i]

                peers = df_by_date[current_date]
                all_mktcap = peers["PRC"].abs().values * peers["SHROUT"].values
                rank = (all_mktcap <= current_mktcap).sum()
                mktcap_percentile = rank / len(all_mktcap)

                # 输入窗口: t-win … t
                X_list.append(exret[i - win + 1 : i + 1])
                # 目标值: t+1
                y_list.append(exret[i + 1])

                meta_list.append(
                    {
                        "PERMNO": permno,
                        "date": signal_date,
                        "ret_date"   : ret_date,
                        "MKTCAP": current_mktcap,
                        "MKTCAP_PERCENTILE": mktcap_percentile,
                    }
                )

        X_arr = np.array(X_list)
        y_arr = np.array(y_list)
        meta_df = pd.DataFrame(meta_list)
        meta_df["signal_month"] = meta_df["date"].dt.to_period("M")
        meta_df["ret_month"]    = meta_df["ret_date"].dt.to_period("M")
        order = meta_df["ret_date"].argsort().values
        X_arr, y_arr = X_arr[order], y_arr[order]
        meta_df = meta_df.iloc[order].reset_index(drop=True)

        results[f"X_{prefix}_{win}"] = X_arr
        results[f"y_{prefix}_{win}"] = y_arr
        results[f"meta_{prefix}_{win}"] = meta_df

        print(
            f" {prefix.upper()} | Window = {win} | X shape = {X_arr.shape} | y shape = {y_arr.shape}"
        )

    return results


def unpack_window_datasets(
    train_datasets: Dict[str, object],
    test_datasets: Dict[str, object],
    window_sizes: List[int] = [5, 21, 252, 512],
):
    for win in window_sizes:
        globals()[f"X_train_{win}"] = train_datasets[f"X_train_{win}"]
        globals()[f"y_train_{win}"] = train_datasets[f"y_train_{win}"]
        globals()[f"meta_train_{win}"] = train_datasets.get(f"meta_train_{win}", None)

        globals()[f"X_test_{win}"] = test_datasets[f"X_test_{win}"]
        globals()[f"y_test_{win}"] = test_datasets[f"y_test_{win}"]
        globals()[f"meta_test_{win}"] = test_datasets.get(f"meta_test_{win}", None)

        print(
            f" Unpacked → X_train_{win}, y_train_{win}, X_test_{win}, y_test_{win}"
        )


def prepare_and_save_scaled_data(
    train_csv_path: str,
    test_csv_path: str,
    window_sizes: List[int] = [5, 21, 252, 512],
    output_npz_path: str = "all_window_datasets_scaled.npz",
    split_date: str = "2016-01-01",
):
    train_df = pd.read_csv(train_csv_path, parse_dates=["date"])
    test_df = pd.read_csv(test_csv_path, parse_dates=["date"])
    full_df = pd.concat([train_df, test_df], ignore_index=True).sort_values(["PERMNO", "date"])

    full_sets = generate_datasets_for_multiple_windows(full_df, window_sizes, "full")
    save_dict = {}

    for win in window_sizes:
        X_full = full_sets[f"X_full_{win}"]
        y_full = full_sets[f"y_full_{win}"]
        meta_full = full_sets[f"meta_full_{win}"]

        mask_train = meta_full["ret_date"] < split_date

        X_train_raw = X_full[mask_train]
        X_test_raw  = X_full[~mask_train]
        y_train_raw = y_full[mask_train].reshape(-1, 1)
        y_test_raw  = y_full[~mask_train].reshape(-1, 1)

        scaler_x = StandardScaler()
        scaler_y = StandardScaler()
        X_train = scaler_x.fit_transform(X_train_raw)
        X_test  = scaler_x.transform(X_test_raw)
        y_train = scaler_y.fit_transform(y_train_raw).flatten()
        y_test  = scaler_y.transform(y_test_raw).flatten()

        meta_train = meta_full[mask_train].reset_index(drop=True)
        meta_test  = meta_full[~mask_train].reset_index(drop=True)

        save_dict[f"X_train_{win}"] = X_train
        save_dict[f"X_test_{win}"]  = X_test
        save_dict[f"y_train_{win}"] = y_train
        save_dict[f"y_test_{win}"]  = y_test
        save_dict[f"meta_train_{win}"] = meta_train.to_dict()
        save_dict[f"meta_test_{win}"] = meta_test.to_dict()

        save_dict[f"market_caps_train_{win}"] = meta_train["MKTCAP"].values
        save_dict[f"market_caps_test_{win}"] = meta_test["MKTCAP"].values

        joblib.dump(scaler_x, f"scaler_X_window_{win}.pkl")
        joblib.dump(scaler_y, f"scaler_y_window_{win}.pkl")

        print(f"[Saved] Window={win} | Train={X_train.shape} | Test={X_test.shape}")

    np.savez(output_npz_path, **save_dict)
    print(f"\n[FINISH] All datasets saved to: {output_npz_path}")


if __name__ == "__main__":
    prepare_and_save_scaled_data(
        train_csv_path="CRSP_2000_2015_top50_with_exret_cleaned.csv",
        test_csv_path="CRSP_2016_2024_top50_with_exret_cleaned.csv",
    )


 FULL | Window = 5 | X shape = (307770, 5) | y shape = (307770,)
 FULL | Window = 21 | X shape = (306970, 21) | y shape = (306970,)
 FULL | Window = 252 | X shape = (295420, 252) | y shape = (295420,)
 FULL | Window = 512 | X shape = (282420, 512) | y shape = (282420,)
[Saved] Window=5 | Train=(196920, 5) | Test=(110850, 5)
[Saved] Window=21 | Train=(196120, 21) | Test=(110850, 21)
[Saved] Window=252 | Train=(184570, 252) | Test=(110850, 252)
[Saved] Window=512 | Train=(171570, 512) | Test=(110850, 512)

[FINISH] All datasets saved to: all_window_datasets_scaled.npz
